Enable Kubernetes in Docker Desktop


Install Azure Functions Core Tools  https://docs.microsoft.com/en-us/azure/azure-functions/functions-run-local?tabs=windows%2Ccsharp%2Cportal%2Cbash%2Ckeda


Requirements:
- Node
- Docker Desktop with Kubernetes enabled
  

In [1]:
!npm i -g azure-functions-core-tools@3 --unsafe-perm true

C:\Program Files\nodejs\func -> C:\Program Files\nodejs\node_modules\azure-functions-core-tools\lib\main.js
C:\Program Files\nodejs\azfun -> C:\Program Files\nodejs\node_modules\azure-functions-core-tools\lib\main.js
C:\Program Files\nodejs\azurefunctions -> C:\Program Files\nodejs\node_modules\azure-functions-core-tools\lib\main.js

> azure-functions-core-tools@3.0.3734 postinstall C:\Program Files\nodejs\node_modules\azure-functions-core-tools
> node lib/install.js

attempting to GET "https://functionscdn.azureedge.net/public/3.0.3734/Azure.Functions.Cli.win-x64.3.0.3734.zip"

Telemetry
---------
The Azure Functions Core tools collect usage data in order to help us improve your experience. 
The data is anonymous and doesn't include any user specific or personal information. The data is collected by Microsoft.

You can opt-out of telemetry by setting the FUNCTIONS_CORE_TOOLS_TELEMETRY_OPTOUT environment variable to '1' or 'true' using your favorite shell.

+ azure-functions-core-tools

In [2]:
!func kubernetes install

Installing KEDA v2 in namespace 
customresourcedefinition.apiextensions.k8s.io/clustertriggerauthentications.keda.sh configured
customresourcedefinition.apiextensions.k8s.io/scaledjobs.keda.sh configured
customresourcedefinition.apiextensions.k8s.io/scaledobjects.keda.sh configured
customresourcedefinition.apiextensions.k8s.io/triggerauthentications.keda.sh configured
serviceaccount/keda-operator unchanged
clusterrole.rbac.authorization.k8s.io/keda-external-metrics-reader unchanged
clusterrole.rbac.authorization.k8s.io/keda-operator configured
rolebinding.rbac.authorization.k8s.io/keda-auth-reader unchanged
clusterrolebinding.rbac.authorization.k8s.io/keda-hpa-controller-external-metrics unchanged
clusterrolebinding.rbac.authorization.k8s.io/keda-operator unchanged
clusterrolebinding.rbac.authorization.k8s.io/keda:system:auth-delegator unchanged
service/keda-metrics-apiserver unchanged
deployment.apps/keda-metrics-apiserver configured
deployment.apps/keda-operator configured
apiservice

- to build folder   func init --docker

In [3]:
!func init fx-edge --docker-only --javascript

Writing Dockerfile
Writing .dockerignore


In [4]:
!func new fx-edge --template "Timer Trigger" --name MyTimerTrigger --worker-runtime node

Writing package.json
Writing .gitignore
Writing host.json
Writing local.settings.json
Writing d:\presentations\azfunc-percept\azfx-k8s-edge\fx-edge\.vscode\extensions.json
Use the up/down arrow keys to select a template:Timer Trigger
Function name: [TimerTrigger] Writing d:\presentations\azfunc-percept\azfx-k8s-edge\fx-edge\MyTimerTrigger\index.js
Writing d:\presentations\azfunc-percept\azfx-k8s-edge\fx-edge\MyTimerTrigger\readme.md
Writing d:\presentations\azfunc-percept\azfx-k8s-edge\fx-edge\MyTimerTrigger\function.json
The function "MyTimerTrigger" was created successfully from the "Timer Trigger" template.


Change the timer to every 15 seconds
```
"schedule": "*/15 * * * * *"
```

Copy local.settings.json for azurerite connection string

In [13]:
!copy azurite/local.settings.json fx-edge/local.settings.json

azurite\azurite.yaml
azurite\local.settings.json
        2 file(s) copied.


In [14]:
!cd fx-edge && docker build --pull --rm -f "Dockerfile" -t rondagdag/fxk8s:latest "."

#1 [internal] load build definition from Dockerfile
#1 sha256:52c38ec3a5b2e9e1da6cdae8ff617d547f7d829c9fe8ec638ee29afea7fb3bdb
#1 transferring dockerfile: 430B done
#1 DONE 0.0s

#2 [internal] load .dockerignore
#2 sha256:92990faec9d0406003e3fe54b9ec07a9b1e01378ee2d400057bb9addb09e0f87
#2 transferring context: 59B done
#2 DONE 0.0s

#3 [internal] load metadata for mcr.microsoft.com/azure-functions/node:3.0
#3 sha256:ab0d5c570b198458f4d0d18cebd5dc49c99be7975c4d134cc1b957f1f7c751a7
#3 DONE 0.7s

#4 [1/3] FROM mcr.microsoft.com/azure-functions/node:3.0@sha256:cc32cdc70a0aa131b7d3f47397eda1341b2ea37a6391b43a439d839f71d449e5
#4 sha256:4a4de030f6687dab9707581a48e618fead7f58045573841261341a2b4c814b3d
#4 DONE 0.0s

#7 [internal] load build context
#7 sha256:cca07ef42e0e1138bab6c60003314561b68bfa71462f6c0b19bbbf4fa724a549
#7 transferring context: 3.09kB done
#7 DONE 0.0s

#5 [2/3] COPY . /home/site/wwwroot
#5 sha256:0c2141aef7f8ec9c38f32affcd86042879edc7c34c1182623a8a0c8f521f127c
#5 CACHED

#6 

In [15]:
!docker push rondagdag/fxk8s:latest

The push refers to repository [docker.io/rondagdag/fxk8s]
8c1066062ef8: Preparing
76135b072dbc: Preparing
9b6823c384b8: Preparing
a9f8bdecd548: Preparing
b0d0c2a63ed8: Preparing
6741d57a517c: Preparing
2c7565da48d9: Preparing
99744128ede5: Preparing
f68ef921efae: Preparing
99744128ede5: Waiting
2c7565da48d9: Waiting
6741d57a517c: Waiting
f68ef921efae: Waiting
8c1066062ef8: Layer already exists
a9f8bdecd548: Layer already exists
76135b072dbc: Layer already exists
9b6823c384b8: Layer already exists
b0d0c2a63ed8: Layer already exists
99744128ede5: Layer already exists
6741d57a517c: Layer already exists
2c7565da48d9: Layer already exists
f68ef921efae: Layer already exists
latest: digest: sha256:e865e14dff53f225cfb2b586f5bc3dbabbc9e98a77451db49597308d8b47490b size: 2219


In [16]:
!kubectl delete storage-azurite-deployment
!kubectl apply -f .\azurite\azurite.yaml

error: the server doesn't have a resource type "storage-azurite-deployment"


deployment.apps/storage-azurite-deployment unchanged
service/storage-azurite-service unchanged


change connection string to "AzureWebJobsStorage": "DefaultEndpointsProtocol=http;AccountName=devstoreaccount1;AccountKey=Eby8vdM02xNOcqFlqUwJPLlmEtlCDXJ1OUzFT50uSRZ6IFsuFq2UVErCz4I6tq/K1SZFPTOtr/KBHBeksoGMGw==;BlobEndpoint=http://storage-azurite-service:10000/devstoreaccount1;QueueEndpoint=http://storage-azurite-service:10001/devstoreaccount1;"

In [17]:
!kubectl delete deployments fx-k8s
!cd fx-edge && func kubernetes deploy --name fx-k8s --image-name rondagdag/fxk8s:latest --dry-run > ../fx-k8s.yaml
!kubectl apply -f fx-k8s.yaml

deployment.apps "fx-k8s" deleted
secret/fx-k8s unchanged
deployment.apps/fx-k8s created
scaledobject.keda.sh/fx-k8s unchanged


In [18]:
!kubectl get services

NAME                      TYPE           CLUSTER-IP       EXTERNAL-IP   PORT(S)                         AGE
fx-k8s-http               LoadBalancer   10.107.122.116   localhost     80:32166/TCP                    170m
kubernetes                ClusterIP      10.96.0.1        <none>        443/TCP                         9h
storage-azurite-service   ClusterIP      10.96.227.211    <none>        10000/TCP,10001/TCP,10002/TCP   60m


In [19]:
!kubectl get pods

NAME                                         READY   STATUS        RESTARTS   AGE
fx-k8s-7895c958d4-55vzx                      0/1     Running       0          15s
fx-k8s-7895c958d4-xq9tq                      1/1     Terminating   0          12m
storage-azurite-deployment-577dbb876-w62gn   1/1     Running       0          60m


In [21]:
!kubectl get deployments

NAME                         READY   UP-TO-DATE   AVAILABLE   AGE
fx-k8s                       1/1     1            1           23s
storage-azurite-deployment   1/1     1            1           60m


In [23]:
!kubectl logs deploy/fx-k8s --tail 20

info: Host.Startup[0]
      Job host started
Hosting environment: Production
Content root path: /
Now listening on: http://[::]:80
Application started. Press Ctrl+C to shut down.
info: Microsoft.Azure.WebJobs.Script.Workers.Rpc.RpcFunctionInvocationDispatcher[0]
      Worker process started and initialized.
info: Function.MyTimerTrigger[1]
      Executing 'Functions.MyTimerTrigger' (Reason='Timer fired at 2021-09-15T05:16:30.0080826+00:00', Id=013290ca-daa3-4470-8dd3-780e192f06a3)
info: Function.MyTimerTrigger.User[0]
      JavaScript timer trigger function ran! 2021-09-15T05:16:30.041Z
info: Function.MyTimerTrigger[2]
      Executed 'Functions.MyTimerTrigger' (Succeeded, Id=013290ca-daa3-4470-8dd3-780e192f06a3, Duration=49ms)
info: Function.MyTimerTrigger[1]
      Executing 'Functions.MyTimerTrigger' (Reason='Timer fired at 2021-09-15T05:16:45.0037562+00:00', Id=19d5ef83-0888-4faf-ade1-149bad71dae6)
info: Function.MyTimerTrigger.User[0]
      JavaScript timer trigger function ran! 202